# One hot enconding of the bedroom info

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [5]:
#setup
df = pd.read_csv("consumption.csv")
weather_avg = pd.read_csv('weather-avg.csv')
weather_min = pd.read_csv('weather-min.csv')
weather_max = pd.read_csv('weather-max.csv')
info = pd.read_csv('addinfo.csv')
weather_avg = weather_avg.set_index("meter_id")
weather_min = weather_min.set_index("meter_id")
weather_max = weather_max.set_index("meter_id")
info = info.set_index("meter_id")
weather_avg_sorted = weather_avg.reindex(df.iloc[:,0])
weather_min_sorted = weather_min.reindex(df.iloc[:,0])
weather_max_sorted = weather_max.reindex(df.iloc[:,0])
info_sorted = info.reindex(df.iloc[:,0])
brinfo=info_sorted['num_bedrooms']
values = {'num_bedrooms' : brinfo.mean()}
info_filled_br = info_sorted.fillna(value = values)
brinfo_filled = info_filled_br['num_bedrooms']
#functions
def get_monthi(n):
    begin=48*31*(n-1)+1
    end=48*31*n
    if n==1:
        begin=1
    if n>1:
        end-=3*48
    if n>2:
        begin-=3*48
    if n>3:
        end-=48
    if n>4:
        begin-=48
    if n>5:
        end-=48
    if n>6:
        begin-=48
    if n>8:
        end-=48
    if n>9:
        begin-=48
    if n>10:
        end-=48
    if n>11:
        begin-=48
    return begin,end

def get_mean_temp(row,month):
    """
    row: is the row (meter_id) we would like to get the average temperature for.
    month: which month (columns) we would get the average temperature for.
    returns: the average temperature for a specific meter_id for a specific month.
    """
    if month==1:
        return row.loc[:,"2017-01-01 00:00:00":"2017-01-31 00:00:00"].mean(1)
    elif month==2:
        return row.loc[:,"2017-02-01 00:00:00":"2017-02-28 00:00:00"].mean(1)
    elif month==3:
        return row.loc[:,"2017-03-01 00:00:00":"2017-03-31 00:00:00"].mean(1)
    elif month==4:
        return row.loc[:,"2017-04-01 00:00:00":"2017-04-30 00:00:00"].mean(1)
    elif month==5:
        return row.loc[:,"2017-05-01 00:00:00":"2017-05-31 00:00:00"].mean(1)
    elif month==6:
        return row.loc[:,"2017-06-01 00:00:00":"2017-06-30 00:00:00"].mean(1)
    elif month==7:
        return row.loc[:,"2017-07-01 00:00:00":"2017-07-31 00:00:00"].mean(1)
    elif month==8:
        return row.loc[:,"2017-08-01 00:00:00":"2017-08-31 00:00:00"].mean(1)
    elif month==9:
        return row.loc[:,"2017-09-01 00:00:00":"2017-09-30 00:00:00"].mean(1)
    elif month==10:
        return row.loc[:,"2017-10-01 00:00:00":"2017-10-31 00:00:00"].mean(1)
    elif month==11:
        return row.loc[:,"2017-11-01 00:00:00":"2017-11-30 00:00:00"].mean(1)
    elif month==12:
        return row.loc[:,"2017-12-01 00:00:00":"2017-12-31 00:00:00"].mean(1)
    else:
        print("Error: this is not a valid input for month")

#Filling missing values with a value outside the range i.e. -1 as a dummy variable:
# Filling the mean value for the NaN values in num_bedrooms column:
values = {'num_bedrooms' : -1}
info_filled_br_dummy = info_sorted.fillna(value = values)
brinfo_filled_dummy = info_filled_br_dummy['num_bedrooms']
brinfo_filled_dummy

meter_id
0xa62b9f23553ff183f61e2bf943aab3d5983d02d7    2.0
0x459c834d1f6cfb5b734b82aa9f5410fa97fb70da   -1.0
0x4a1ed36825360a058cec2bdd409fc2459e1ce54f   -1.0
0x5b76d3c0e0aefc6e0a8d1d031f96388a23263407   -1.0
0x943ebe39ef2be6ef807c42c5a647e27112ca5b0f   -1.0
                                             ... 
0x7dd7a7b8ee1bec7c44b24f738c752482f6161065   -1.0
0xfdaf9f857621ec06f2cf801f42a020a322835090   -1.0
0xd28f2f001e0cd4d6c121a3cb2e1427207e170e18   -1.0
0x47218b46abb2fcaade487a211911406dc6e13730   -1.0
0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd   -1.0
Name: num_bedrooms, Length: 3248, dtype: float64

In [21]:
def sort_data(nim):
    NaN_t=1200
    En_con=[]
    #number of input months (if nim=1 we should get the same as before)
    #this is the amount of months we want in our input space to predict the next monthly consumption     
    prev_con=[] #previous consumtion
    #extra info
    temps=[]
    temps_min=[]
    temps_max=[]
    month_arr=[]
    br_arr=[]
    br_arr_dummy=[]
    dwelling_type_bungalow=[]
    dwelling_type_detached_house=[]
    dwelling_type_flat=[]
    dwelling_type_semi_detached_house=[]
    dwelling_type_terraced_house=[]
    y = pd.get_dummies(info_sorted.dwelling_type, prefix='dwelling_type')
    
    
    for i in range(df.shape[0]): # loop over all users
        meter=df.iloc[i] # = row number i  
    
        #loop over all month from januari until month 12-nim
        #if the month is higher then this we don't have enough data for the nim input variables and the 1 output variable
        for m in range (1,13-nim): 
            #load current month
            bi, ei =get_monthi(m)               #bi (begin index) ei(end index)
            month=meter[bi:ei]  #data for the month m for the row user (row) i
            #load bedroom info
            row = brinfo[i:i+1]
            row_dummy = brinfo_filled_dummy[i:i+1]
            #Load dwelling type
            dew_type_row = y[i:i+1]
            
            # Count the amount of NaN's in current      
            n_NaN=month.isnull().sum() 

            # Check if months have numeric values otherwise discard the month for this user. 
            if n_NaN<NaN_t:
                for j in range(m,13-nim):
                    #save input for month j,j+1,..,j+nim-1
                    input_months=np.zeros(nim)
                    for k in range(nim):
                        #load month for input j+k
                        bik,eik=get_monthi(j+k)
                        input_months[k]=meter[bik:eik].mean()
                    #debug if statement checks for NaN in input month
                    if np.isnan(np.sum(input_months)):
                        print("months=%i index=%i" %(j,i))
                        print(input_months)
                    prev_con.append(input_months)
                    #save output for month j+nim
                    bio,eio=get_monthi(j+nim)
                    En_con.append(meter[bio:eio].mean())
                    #save extra information 
                    temps.append(get_mean_temp(weather_avg_sorted.iloc[i:i+1],j+nim))
                    temps_min.append(get_mean_temp(weather_min_sorted.iloc[i:i+1],j+nim))
                    temps_max.append(get_mean_temp(weather_max_sorted.iloc[i:i+1],j+nim))
                    month_arr.append(j+nim)
                    #one hot encoding dwelling type
                    dwelling_type_bungalow.append(dew_type_row.iloc[0, 0])
                    dwelling_type_detached_house.append(dew_type_row.iloc[0, 1])
                    dwelling_type_flat.append(dew_type_row.iloc[0, 2])
                    dwelling_type_semi_detached_house.append(dew_type_row.iloc[0, 3])
                    dwelling_type_terraced_house.append(dew_type_row.iloc[0, 4])
                    #one hot encoding bedroom info
                    n_br=row[0]
                    #if n_br==1:
                    #    br=[1, 0, 0, 0, 0]
                    #elif n_br==2:
                    #    br=[0, 1, 0, 0, 0]
                    #elif n_br==3:
                    #    br=[0, 0, 1, 0, 0]
                    #elif n_br==4:
                    #    br=[0, 0, 0, 1, 0]
                    #elif n_br==5:
                    #    br=[0, 0, 0, 0, 1]
                    #else:
                        #Only the NaN should end up here
                    #    br=[0, 0, 0, 0]
                    #    if ~np.isnan(n_br):
                    #        print('A non NaN values ended up in else category of br info one hot encoding. n_br=%i' %n_br)
                    br=np.zeros(5)
                    if ~np.isnan(n_br):
                        #print(n_br)
                        br[np.int(n_br-1)]=1
                    br_arr.append(br)
                        
                            

                    
                
                
                #break out of outer month loops since the inner month loop takes care of all months
                break
    # converting list to numpy arrays: 
    nptemps = np.zeros(len(temps))
    nptemps_min = np.zeros(len(temps_min))
    nptemps_max = np.zeros(len(temps_max))
    npEn_con = np.zeros(len(En_con))
    npmonth_arr = np.zeros(len(month_arr))
    #npbr_arr = np.zeros(len(br_arr)) #converting it into numpy array.
    npbr_arr_dummy = np.zeros(len(br_arr_dummy)) #converting it into numpy array.

    for i in range(len(temps)):
         nptemps[i] = temps[i]

    for i in range(len(temps_min)):
         nptemps_min[i] = temps_min[i]

    for i in range(len(temps_max)):
         nptemps_max[i] = temps_max[i]

    for i in range(len(En_con)):
         npEn_con[i] = En_con[i]

    for i in range(len(month_arr)):
         npmonth_arr[i] = month_arr[i]   
    
    for i in range(len(br_arr_dummy)):
        npbr_arr_dummy[i] = br_arr_dummy[i]
        
    npprev_con=np.array(prev_con)
    npbr_arr=np.array(br_arr)
    dw_t_b = np.array(dwelling_type_bungalow)
    dw_t_d = np.array(dwelling_type_detached_house)
    dw_t_f = np.array(dwelling_type_flat)
    dw_t_s = np.array(dwelling_type_semi_detached_house)
    dw_t_t = np.array(dwelling_type_terraced_house)
    
    #This should probably be done differently 
    return npEn_con,npprev_con,nptemps,nptemps_min,nptemps_max,npmonth_arr,npbr_arr,npbr_arr_dummy,dw_t_b,dw_t_d,dw_t_f,dw_t_s,dw_t_t
    

In [22]:
#nim=1
npEn_con1,npprev_con1,nptemps1,nptemps_min1,nptemps_max1,npmonth_arr1,npbr_arr1,npbr_arr_dummy1,dw_t_b1,dw_t_d1,dw_t_f1,dw_t_s1,dw_t_t1=sort_data(1)
#nim=2
npEn_con2,npprev_con2,nptemps2,nptemps_min2,nptemps_max2,npmonth_arr2,npbr_arr2,npbr_arr_dummy2,dw_t_b2,dw_t_d2,dw_t_f2,dw_t_s2,dw_t_t2=sort_data(2)


In [31]:
def Eval_Kfold_Split(features,npEn_con,n_splits,model,print_res=True):
    kf = KFold(n_splits, shuffle=True)
    models_arr=[]
    MSE_arr=[]
    R2_arr=[]
    for train_index, test_index in kf.split(features): 
        cmodel=model
        cmodel.fit(features[train_index,],npEn_con[train_index])
        models_arr.append(cmodel)
        y_pred = model.predict(features[test_index,])
        MSE_arr.append(mean_squared_error(npEn_con[test_index,], y_pred));
        R2_arr.append(r2_score(npEn_con[test_index,], y_pred))
    if print_res:
        # The mean squared error
        print('Mean squared error:')
        print(np.mean(MSE_arr))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination: ' )
        print(np.mean(R2_arr))
    return models_arr
#z-normalization on data:
def normalise_features(np_features):
    #np_features_norm= np.zeros(shape = np_features.shape)
    np_features_norm=np_features
    if np_features.ndim==1:
        np_features_norm=(np_features-np.mean(np_features))/np.std(np_features)
    else: 
        for i in range(np_features.shape[1]):
            np_features_norm[:,i] = (np_features[:,i]-np.mean(np_features[:,i]))/np.std(np_features[:,i])
    return np_features_norm

In [29]:
#1 month
print("1 months")
np_features=np.column_stack(((npprev_con1,nptemps1,nptemps_min1, nptemps_max1, npbr_arr1)))
models_arr=Eval_Kfold_Split(np_features,npEn_con1,10,LinearRegression())
print("\n")
#2 months
print("2 months")
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2, npbr_arr2)))
models_arr=Eval_Kfold_Split(np_features,npEn_con2,10,LinearRegression())

1 months
Mean squared error:
0.0031398296814709923
Coefficient of determination: 
0.8433754909243805


2 months
Mean squared error:
0.0029469970190887677
Coefficient of determination: 
0.8518187804445873


In [32]:
#extra features
np_features=np.column_stack(((npprev_con1,nptemps1,nptemps_min1, nptemps_max1, npbr_arr1,dw_t_b1,dw_t_d1,dw_t_f1,dw_t_s1,dw_t_t1)))
print("1 months")
np_features=np.column_stack(((npprev_con1,nptemps1,nptemps_min1, nptemps_max1, npbr_arr1)))
models_arr=Eval_Kfold_Split(np_features,npEn_con1,10,LinearRegression())
print("normalised")
np_features_norm=normalise_features(np_features)
npEn_con1_norm=normalise_features(npEn_con1)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con1_norm,10,LinearRegression())
print("\n")
print("2 months")
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2, npbr_arr2,dw_t_b2,dw_t_d2,dw_t_f2,dw_t_s2,dw_t_t2)))
models_arr=Eval_Kfold_Split(np_features,npEn_con2,10,LinearRegression())
print("normalised")
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_Kfold_Split(np_features_norm,npEn_con2_norm,10,LinearRegression())

1 months
Mean squared error:
0.0031378049011288336
Coefficient of determination: 
0.8429258866714784
normalised
Mean squared error:
0.15578789006803984
Coefficient of determination: 
0.8436556043503621


2 months
Mean squared error:
0.0029450267714494225
Coefficient of determination: 
0.852545584791194
normalised
Mean squared error:
0.14689452943011422
Coefficient of determination: 
0.8529174001595143


## K nearest neighbors

In [42]:
def Eval_random_Split(features,npEn_con,split,model,print_res=True):
    X_train, X_test, y_train, y_test = train_test_split(
    features , npEn_con.reshape(-1,1), test_size=split, random_state=0)
    #print(X_train.shape, y_train.shape)
    #print(X_test.shape, y_test.shape)
    model.fit(X_train,y_train)
    if print_res:
        y_pred=model.predict(X_test)
        #print('Coefficients: \n', model.coef_)
        #print('Intercept: {}'.format(model.intercept_))
        # The mean squared error
        print('Mean squared error: %.8f' % mean_squared_error(y_test, y_pred))
        # The coefficient of determination: 1 is perfect prediction
        print('Coefficient of determination: %.8f' % r2_score(y_test, y_pred))
    return model

In [71]:
from sklearn.neighbors import KNeighborsRegressor
model= KNeighborsRegressor(n_neighbors=15)
np_features=npprev_con2 #this already works okay
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_random_Split(np_features_norm,npEn_con2_norm,0.3,model)

Mean squared error: 0.14547566
Coefficient of determination: 0.84997621


In [73]:
model= KNeighborsRegressor(n_neighbors=15)
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2)))
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_random_Split(np_features_norm,npEn_con2_norm,0.3,model)

Mean squared error: 0.13613700
Coefficient of determination: 0.85960682


In [78]:
model= KNeighborsRegressor(n_neighbors=15)
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2,npmonth_arr2)))
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_random_Split(np_features_norm,npEn_con2_norm,0.3,model)

Mean squared error: 0.13177813
Coefficient of determination: 0.86410197


In [96]:
model= KNeighborsRegressor(n_neighbors=30,weights='distance')
np_features=np.column_stack(((npprev_con2,nptemps2,nptemps_min2, nptemps_max2,npmonth_arr2)))
np_features_norm=normalise_features(np_features)
npEn_con2_norm=normalise_features(npEn_con2)
models_arr=Eval_random_Split(np_features_norm,npEn_con2_norm,0.3,model)

Mean squared error: 0.13335812
Coefficient of determination: 0.86247258
